In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.chrome.options import Options  


In [2]:
chrome_options = Options()  
chrome_options.add_argument("--headless")  
chrome_options.add_argument("--no-sandbox")

driver = webdriver.Chrome('chromedriver', options=chrome_options)

### Extract date



In [3]:
url = 'https://www.moh.gov.sg/covid-19'

driver.get(url)
content = driver.page_source


In [4]:
soup =BeautifulSoup(content, 'lxml')

In [5]:
data = soup.findAll(text=re.compile("Case Summary in Singapore \(as of .+"))


In [6]:
data

['Case Summary in Singapore (as of 19 Apr 2020, 1200h)']

In [7]:
m = re.search("(\d+ \w+ 202\d, \d+\d+h)", data[0])
date_time_str = m.group(1)
data_date_time = datetime.strptime(date_time_str, '%d %b %Y, %H%Mh').strftime('%Y-%m-%d')
data_date_time


'2020-04-19'

### Extract Dataframes

In [8]:
dfs = pd.read_html(content)

In [9]:
dfs

[    0   1
 0 NaN NaN
 1 NaN NaN,
     0   1
 0 NaN NaN
 1 NaN NaN,
                0       1
 0  DORSCON Level  Orange,
           0         1         2
 0  IMPORTED  IMPORTED  IMPORTED
 1  569 (+0)  569 (+0)  569 (+0),
                0
 0  Active Cases#
 1           2921,
              0
 0  Discharged^
 1          768,
                          0
 0  Discharge to Isolation*
 1                     2888,
                        0
 0  Hospitalised (Stable)
 1                   2899,
                          0
 0  Hospitalised (Critical)
 1                       22,
         0
 0  Deaths
 1      11,
        0
 0  94796,
        0
 0  59737,
          0
 0  ~16,600,
          0
 0  ~10,500,
               0                                                  1
 0          Date                                              Title
 1   20 Apr 2020  Condominium Residents Fined for Flouting Covid...
 2   20 Apr 2020  Two Singapore Citizens And One Foreigner To Be...
 3   19 Apr 2020  Temporary 

In [10]:
df_idx = 0
for idx in range(len(dfs)):
    if 'Active' in str(dfs[idx][0][0]):
        print("found starting df")
        df_idx = idx
        break
print(df_idx)
assert df_idx < len(dfs)

found starting df
4


In [11]:
dfs[df_idx]

,0
0,Active Cases#
1,2921


In [12]:
active=df_idx
assert dfs[active][0][0].strip().startswith('Active Cases')
dfs[active]


,0
0,Active Cases#
1,2921


In [13]:
discharge = df_idx+1
assert dfs[discharge][0][0].strip().startswith('Discharged')
dfs[discharge]

,0
0,Discharged^
1,768


In [14]:
hospitalised_stable = df_idx+3
assert dfs[hospitalised_stable][0][0].strip().startswith('Hospitalised (Stable)')

dfs[hospitalised_stable]

,0
0,Hospitalised (Stable)
1,2899


In [15]:
hospitalised_critical = df_idx+4
assert dfs[hospitalised_critical][0][0].strip().startswith('Hospitalised (Critical)')

dfs[hospitalised_critical]

,0
0,Hospitalised (Critical)
1,22


In [16]:
deaths = df_idx+5
assert dfs[deaths][0][0].strip().startswith('Deaths')

dfs[deaths]

,0
0,Deaths
1,11


In [17]:
# # Look for df with confirmed cases, tested negative, ...

# for df in dfs:
#     combined_fields = df[3].to_string().lower()
#     if "active cases" in combined_fields:
#         sing_cases_df_temp = df
        
# sing_cases_df_temp

In [18]:
def remove_non_numeric(s):
    return re.sub('\D','', s)


In [19]:
# very hacky to workaround with the change from MoH site
sing_cases_df = pd.DataFrame({'active_cases': [int(remove_non_numeric(dfs[active][0][1]))],
                             'hospitalised_stable': [int(remove_non_numeric(dfs[hospitalised_stable][0][1]))],
                              'hospitalised_critical': [int(remove_non_numeric(dfs[hospitalised_critical][0][1]))],
                              'death': [int(remove_non_numeric(dfs[deaths][0][1]))],
                              'discharge': [int(dfs[discharge][0][1])],
                              'datetime': data_date_time
                             },
                            )
sing_cases_df

,active_cases,hospitalised_stable,hospitalised_critical,death,discharge,datetime
0,2921,2899,22,11,768,2020-04-19


In [20]:
df = sing_cases_df
df

,active_cases,hospitalised_stable,hospitalised_critical,death,discharge,datetime
0,2921,2899,22,11,768,2020-04-19


### Save to file

In [21]:
from pathlib import Path
csv_file = '../data/singapore-cases.csv'

if Path(csv_file).exists():
    # read out the old data
    old_df = pd.read_csv(csv_file)
    df = df.set_index('datetime').append(old_df.set_index('datetime'))



/opt/conda/lib/python3.7/site-packages/pandas/core/frame.py:7138: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,


In [22]:
df = df.reset_index()
df.drop_duplicates(subset=['datetime'], inplace=True)

In [23]:
df.sort_values(by=['datetime'], inplace=True)
df

,datetime,active_cases,confirmed,death,discharge,discharged,hospitalised,hospitalised_critical,hospitalised_stable,negative,pending
1,2020-02-02,NaN,18.0,NaN,NaN,NaN,NaN,NaN,NaN,240.0,43.0
2,2020-02-03,NaN,24.0,NaN,NaN,NaN,NaN,NaN,NaN,262.0,32.0
3,2020-02-04,NaN,24.0,NaN,NaN,NaN,NaN,NaN,NaN,289.0,20.0
4,2020-02-05,NaN,28.0,NaN,NaN,NaN,NaN,NaN,NaN,295.0,62.0
5,2020-02-06,NaN,33.0,NaN,NaN,NaN,NaN,NaN,NaN,310.0,147.0
...,...,...,...,...,...,...,...,...,...,...,...
74,2020-04-15,1496.0,NaN,10.0,653.0,NaN,NaN,26.0,1470.0,NaN,NaN
75,2020-04-16,1886.0,NaN,10.0,683.0,NaN,NaN,23.0,1863.0,NaN,NaN
76,2020-04-17,2113.0,NaN,11.0,708.0,NaN,NaN,22.0,2091.0,NaN,NaN
77,2020-04-18,2563.0,NaN,11.0,740.0,NaN,NaN,23.0,2540.0,NaN,NaN


In [24]:
df.to_csv(csv_file, index=False)
